In [2]:
import javalang
import os
import pandas as pd
import time

In [3]:
d={'class_name':[], 'methods_num':[]}
dataframe= pd.DataFrame(data=d)

In [4]:
start=time.time()
import numpy as np
array_of_paths={'name':[], 'path':[], 'node':[]}
for root, dirs, files in os.walk(".", topdown=False):
    
    for name in files:
      
      if(name.endswith(".java")):
        
        nome=os.path.join(root, name)
        
        data = open(nome).read()
        tree_tmp=javalang.parse.parse(data)
        
        for klass in tree_tmp.types:
            if isinstance(klass, javalang.tree.ClassDeclaration) and klass.name is not d['class_name']:
                name_class=klass.name
                array_of_paths['name'].append(name_class)
                array_of_paths['path'].append(nome)
                array_of_paths['node'].append(klass)
                d['class_name'].append(name_class)
                d['methods_num'].append(len(klass.methods))
print("time of execution: ", time.time()-start, " seconds")

time of execution:  11.180932760238647  seconds


In [5]:
dataframe=pd.DataFrame(d)
print(dataframe)
mean_num_methods=dataframe['methods_num'].mean()
std_num_methods=dataframe['methods_num'].std()
print("mean of methods= ", dataframe['methods_num'].mean())
print("std of methods= ", dataframe['methods_num'].std())

names_god=dataframe.loc[dataframe['methods_num']>(mean_num_methods+6*(std_num_methods))]

#print(names)


                  class_name  methods_num
0        HTMLAreaElementImpl           16
1      HTMLLegendElementImpl            4
2    HTMLFieldSetElementImpl            0
3         HTMLModElementImpl            4
4         HTMLMapElementImpl            4
..                       ...          ...
556     WMLAnchorElementImpl            8
557      WMLTimerElementImpl            8
558       WMLHeadElementImpl            4
559          WMLBElementImpl            6
560     WMLAccessElementImpl            8

[561 rows x 2 columns]
mean of methods=  12.035650623885918
std of methods=  15.941594683379295


In [6]:
print("the God classes are: ")
for i in names_god["class_name"]:
    print(i, end='\n')




the God classes are: 
XSDHandler
XIncludeHandler
CoreDocumentImpl


In [7]:
god_class_names=pd.DataFrame(data=array_of_paths)
#print(god_class_names['name'])
#print(names_god, end='\n******************')
god_class_names_final=god_class_names.loc[god_class_names['name'].isin(names_god['class_name'])]

print(god_class_names_final)


                 name                                               path  \
128        XSDHandler  ./resources/xerces2-j-src/org/apache/xerces/im...   
307   XIncludeHandler  ./resources/xerces2-j-src/org/apache/xerces/xi...   
452  CoreDocumentImpl  ./resources/xerces2-j-src/org/apache/xerces/do...   

                                                  node  
128  ClassDeclaration(annotations=[], body=[FieldDe...  
307  ClassDeclaration(annotations=[], body=[FieldDe...  
452  ClassDeclaration(annotations=[], body=[FieldDe...  


In [8]:

def get_methods(java_class_):
    array_of_methods=[]
    for i in java_class_.methods:
        array_of_methods.append(i.name)
    return (array_of_methods)

def get_fields(java_class_):
    array_of_fields=[]
    for i in java_class_.fields:
        for k in i.declarators:
            #print(k)
            if k.name not in array_of_fields:
                array_of_fields.append(k.name)
        
    return (array_of_fields)

def get_fields_accessed_by_method(method, fields):
    #print(method)
    tree=method
    array_of_fields_of_this_method=[]
    for i, node in tree.filter(javalang.tree.MemberReference):
        #print(node.member)
        if node.member not in fields and node.member not in array_of_fields_of_this_method:
            array_of_fields_of_this_method.append(node.member)
    return array_of_fields_of_this_method



def get_methods_accessed_by_method(method, methods):
    tree=method
    array_of_methods_of_this_method=[]
    for i, node in tree.filter(javalang.tree.MethodInvocation):
        #print(node.member)
        if( node.member in methods and node.member !=method.name):
            array_of_methods_of_this_method.append(node.member)
        #if node.member not in fields and node.member not in array_of_fields_of_this_method:
        #    array_of_fields_of_this_method.append(node.member)
    return array_of_methods_of_this_method

java_god_class=god_class_names_final['node'].values[2]
#print(get_methods(java_god_class))
#print(god_class_names_final['path'].values[0])
#print(java_god_class.fields)
#print(get_fields(java_god_class))
#print(len(get_fields(java_god_class)))

columns=np.asarray(((get_fields(java_god_class)), get_methods(java_god_class)))
#print(columns)
frame_final={}
all_methods=get_methods(java_god_class)
all_fields=get_fields(java_god_class)

for i in all_methods:
    frame_final[i]=np.int64(np.zeros(len(all_methods)))
for i in all_fields:
    frame_final[i]=np.int64(np.zeros(len(all_methods)))
frame_final=pd.DataFrame(frame_final)

for i in (java_god_class.methods):
   
    fields_accessed_by_method=get_fields_accessed_by_method(i, get_fields(java_god_class))
    methods_accessed_by_method=get_methods_accessed_by_method(i, get_methods(java_god_class))
    row=0
    for n in frame_final.columns:
        #print(n)
        if(n in fields_accessed_by_method):
            frame_final.at[n, row] = 1
        if(n in methods_accessed_by_method):
            frame_final.at[n, row]=1
        row+=1

        
print(frame_final.info())
print(frame_final.std())
frame_final.to_csv("./"+java_god_class.name + ".csv")




    
        

#print(frame_final)




<class 'pandas.core.frame.DataFrame'>
Index: 166 entries, 0 to removeIdentifier
Columns: 180 entries, getOwnerDocument to 68
dtypes: float64(180)
memory usage: 238.8+ KB
None
getOwnerDocument    0.0
getNodeType         0.0
getNodeName         0.0
cloneNode           0.0
insertBefore        0.0
                   ... 
3                   NaN
63                  NaN
83                  NaN
67                  NaN
68                  NaN
Length: 180, dtype: float64


/var/folders/4w/9ln2nw453jxggvcfqdqdkclw0000gn/T/ipykernel_14312/620149150.py:47: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  columns=np.asarray(((get_fields(java_god_class)), get_methods(java_god_class)))
